In [1]:
import pandas as pd

In [2]:
df2020 = pd.read_csv('/home/ubuntu/workspace/vision.pjt/train.csv')
df2019 = pd.read_csv('/home/ubuntu/workspace/vision.pjt/data/ISIC_2019_Training_GroundTruth.csv')

In [3]:
df2020

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
...,...,...,...,...,...,...,...,...
33121,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0
33122,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0
33123,ISIC_9999515,IP_2026598,male,20.0,lower extremity,unknown,benign,0
33124,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0


In [6]:
df2020.groupby('diagnosis').count()['image_name']

diagnosis
atypical melanocytic proliferation        1
cafe-au-lait macule                       1
lentigo NOS                              44
lichenoid keratosis                      37
melanoma                                584
nevus                                  5193
seborrheic keratosis                    135
solar lentigo                             7
unknown                               27124
Name: image_name, dtype: int64

In [4]:
df2019

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
img2019, diag2019 = list(), list()
for idx, row in df2019.iterrows():
    if row['MEL'] == 1.0:
        img2019.append(row['image'])
        diag2019.append('melanoma')
    elif row['NV'] == 1.0:
        img2019.append(row['image'])
        diag2019.append('nevus')
    elif row['BKL'] == 1.0:
        img2019.append(row['image'])
        diag2019.append('bkl')
    elif row['UNK'] == 1.0:
        img2019.append(row['image'])
        diag2019.append('unknown')

In [15]:
img2020, diag2020 = list(), list()
for idx, row in df2020.iterrows():
    diagnosis = row['diagnosis']
    if diagnosis == 'melanoma':
        img2020.append(row['image_name'])
        diag2020.append('melanoma')
    elif diagnosis == 'nevus':
        img2020.append(row['image_name'])
        diag2020.append('nevus')
    elif diagnosis in ['seborrheic keratosis', 'solar lentigo', 'lentigo NOS', 'lichenoid keratosis']:
        img2020.append(row['image_name'])
        diag2020.append('bkl')
    elif diagnosis in ['unknown', 'cafe-au-lait macule', 'atypical melanocytic proliferation']:
        img2020.append(row['image_name'])
        diag2020.append('unknown')

In [16]:
skin_combine = pd.DataFrame({'image_name': img2019+img2020, 'diagnosis':diag2019+diag2020})
skin_combine

,image_name,diagnosis
0,ISIC_0000000,nevus
1,ISIC_0000001,nevus
2,ISIC_0000002,melanoma
3,ISIC_0000003,nevus
4,ISIC_0000004,melanoma
...,...,...
53142,ISIC_9999134,unknown
53143,ISIC_9999320,unknown
53144,ISIC_9999515,unknown
53145,ISIC_9999666,unknown


In [29]:
cnt = skin_combine.groupby('diagnosis').count().reset_index()
cnt['percent'] = cnt['image_name'].map(lambda x: float(x/cnt['image_name'].sum()) * 100)
cnt

,diagnosis,image_name,percent
0,bkl,2847,5.356840
1,melanoma,5106,9.607316
2,nevus,18068,33.996274
3,unknown,27126,51.039569


In [30]:
skin_combine.to_csv('../skin_combine.csv', index=False)